##### This Project is all about the generation of synthetic datasets in variational autoencoders and the crop dataset what will be used as experiment here

In [105]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Conv2DTranspose, UpSampling2D, Layer
from keras.models import Model,Sequential
from keras.losses import MSE
from keras.callbacks import EarlyStopping
from keras import backend as K
import numpy as np
import keras

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('crop modified.csv')
df.head(5)

,Unnamed: 0,N,P,K,temperature,humidity,ph,rainfall,label
0,0,90,42,43,20.879744,82.002744,6.502985,202.935536,20
1,1,85,58,41,21.770462,80.319644,7.038096,226.655537,20
2,2,60,55,44,23.004459,82.320763,7.840207,263.964248,20
3,3,74,35,40,26.491096,80.158363,6.980401,242.864034,20
4,4,78,42,42,20.130175,81.604873,7.628473,262.717340,20


In [3]:
df2 = df.drop(['label', 'Unnamed: 0'], axis='columns')
df2

,N,P,K,temperature,humidity,ph,rainfall
0,90,42,43,20.879744,82.002744,6.502985,202.935536
1,85,58,41,21.770462,80.319644,7.038096,226.655537
2,60,55,44,23.004459,82.320763,7.840207,263.964248
3,74,35,40,26.491096,80.158363,6.980401,242.864034
4,78,42,42,20.130175,81.604873,7.628473,262.717340
...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507
2196,99,15,27,27.417112,56.636362,6.086922,127.924610
2197,118,33,30,24.131797,67.225123,6.362608,173.322839
2198,117,32,34,26.272418,52.127394,6.758793,127.175293


In [4]:
# Scale the data so they can be on the same scale all through
scaler = StandardScaler()
x_scaled = scaler.fit_transform(df2)

In [5]:
input_dim = x_scaled.shape[1]
input_dim

7

##### Building the AutoEncoder Architecture

In [169]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + mean

In [184]:
encoder_input = Input(shape=(input_dim,))
x = Dense(10, activation='relu')(encoder_input)
x = Dense(5, activation='relu')(x)
z_mean = Dense(2)(x)
z_log_var = Dense(2)(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.models.Model(inputs=[encoder_input], outputs=[z_mean, z_log_var, z],name="encoder")


In [185]:
decoder_input = Input(shape=2,)
x = Dense(5, activation='relu')(decoder_input)
x = Dense(10, activation='relu')(x)
decoder_output = Dense(7)(x)
decoder = keras.models.Model(inputs=[decoder_input], outputs=[decoder_output])

In [186]:
_, _, codings = encoder(encoder_input)
reconstructions = decoder(codings)
variational_ae = tf.keras.Model(inputs=[encoder_input], outputs=[reconstructions])

In [187]:
latent_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.exp(z_log_var) - tf.square(z_mean), axis=-1)
variational_ae.add_loss(tf.reduce_mean(latent_loss)/7)

In [188]:
variational_ae.compile(loss='mse', optimizer='nadam')

In [189]:
variational_ae.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_21 (InputLayer)       [(None, 7)]                  0         []                            
                                                                                                  
 encoder (Functional)        [(None, 2),                  159       ['input_21[0][0]']            
                              (None, 2),                                                          
                              (None, 2)]                                                          
                                                                                                  
 model_20 (Functional)       (None, 7)                    152       ['encoder[0][2]']             
                                                                                           

In [142]:
data = df.drop('Unnamed: 0', axis='columns')
data

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,20
1,85,58,41,21.770462,80.319644,7.038096,226.655537,20
2,60,55,44,23.004459,82.320763,7.840207,263.964248,20
3,74,35,40,26.491096,80.158363,6.980401,242.864034,20
4,78,42,42,20.130175,81.604873,7.628473,262.717340,20
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,5
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,5
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,5
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,5


In [143]:
input_dim = x_scaled.shape[1]
input_dim

7

In [144]:
x = data.drop('label', axis='columns')

In [145]:
y = data['label']

In [146]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [147]:
x_train.shape

(1760, 7)

In [148]:
x_test.shape

(440, 7)

In [149]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

In [190]:
history = variational_ae.fit(x_train, x_train, epochs=100, verbose=False, validation_data=(x_test, x_test))

In [191]:
codings_synthetic = tf.random.normal(shape=[300 * 7, 2])
synthetic_data = decoder(codings_synthetic).numpy()

In [192]:
synthetic_data.shape

(2100, 7)

In [193]:
synthetic_data

array([[ 7.8330956 ,  9.671045  , 14.2806835 , ...,  9.496633  ,
         1.8481503 , 19.115318  ],
       [ 1.959469  ,  4.475361  ,  7.4609694 , ...,  2.4634042 ,
         0.47455043,  5.2739835 ],
       [15.689408  , 12.3034935 , 12.958933  , ..., 17.077501  ,
         1.485981  , 31.573254  ],
       ...,
       [ 9.168924  ,  6.4695554 ,  7.3531303 , ...,  8.796936  ,
         1.3015947 , 16.976982  ],
       [10.869218  ,  7.903859  ,  8.710179  , ..., 10.909619  ,
         1.3118246 , 20.714443  ],
       [ 6.803891  ,  6.3711486 ,  9.757695  , ...,  6.452675  ,
         1.5314162 , 14.440849  ]], dtype=float32)

In [194]:
synthetic_dataframe = pd.DataFrame(synthetic_data, columns=['sN', 'sP', 'sK', 'sTemperature', 'sHumidity', 'sPh', 'sRainfall'])
synthetic_dataframe

,sN,sP,sK,sTemperature,sHumidity,sPh,sRainfall
0,7.833096,9.671045,14.280684,4.564231,9.496633,1.848150,19.115318
1,1.959469,4.475361,7.460969,0.789409,2.463404,0.474550,5.273983
2,15.689408,12.303493,12.958933,7.084591,17.077501,1.485981,31.573254
3,13.680435,10.605597,11.689374,6.212321,14.418177,1.476945,27.324600
4,12.642057,9.411396,10.085269,5.468603,13.129207,1.351490,24.588264
...,...,...,...,...,...,...,...
2095,3.235728,3.946060,7.453334,1.911408,2.572058,1.289479,7.158320
2096,9.802545,9.975166,13.659416,5.278688,10.937067,1.822081,21.941944
2097,9.168924,6.469555,7.353130,3.777588,8.796936,1.301595,16.976982
2098,10.869218,7.903859,8.710179,4.612666,10.909619,1.311825,20.714443


In [61]:
x

,N,P,K,temperature,humidity,ph,rainfall
0,90,42,43,20.879744,82.002744,6.502985,202.935536
1,85,58,41,21.770462,80.319644,7.038096,226.655537
2,60,55,44,23.004459,82.320763,7.840207,263.964248
3,74,35,40,26.491096,80.158363,6.980401,242.864034
4,78,42,42,20.130175,81.604873,7.628473,262.717340
...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507
2196,99,15,27,27.417112,56.636362,6.086922,127.924610
2197,118,33,30,24.131797,67.225123,6.362608,173.322839
2198,117,32,34,26.272418,52.127394,6.758793,127.175293
